In [1]:
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
import requests
from PIL import Image
from io import BytesIO

### Text2Image flow

In [ ]:

#model_id = "CompVis/stable-diffusion-v1-4"
model_id = "runwayml/stable-diffusion-v1-5"
device = "cuda"

# this for the regular FP32 model
#pipe = StableDiffusionPipeline.from_pretrained(model_id, use_auth_token=True)
#pipe = pipe.to(device)

#this for the FP16 model
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16, revision="fp16", use_auth_token=True)
pipe = pipe.to(device)

In [ ]:
prompt = "a photo of an astronaut riding a horse on mars"
batch_size = 1
prompt_input = [prompt] * batch_size

with autocast("cuda"):
    image = pipe(prompt_input, guidance_scale=7.5).images[0]
    
#image.save("astronaut_rides_horse.png")

image

In [ ]:
# this is to test out how to convert to base64, to pass in using to the managed endpoint
from io import BytesIO
import base64

result_image_str = [""] * 3
print(len(result_image_str))

for i in range(len(images)):

    buffered_result_image = BytesIO()

    images[i].save(buffered_result_image, format="PNG")
    result_image_str[i] = base64.b64encode(buffered_result_image.getvalue()).decode('utf-8')

result_image_str

In [ ]:
#this is how you save the model to register on the aml workspace
pipe.save_pretrained('models/stable-diffusion-v1-5')

### Image2Image Flow

In [2]:
model_id = "CompVis/stable-diffusion-v1-4"
#model_id = "runwayml/stable-diffusion-v1-5"
device = "cuda"


#this for the FP16 model
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, torch_dtype=torch.float16, revision="fp16", use_auth_token=True)
pipe = pipe.to(device)

text_encoder/model.safetensors not found
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
/anaconda/envs/sd_xl/lib/python3.9/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


In [3]:
from PIL import Image
import os

file =  os.path.join("images", "elmo.jpg")
img = Image.open(file).convert("RGB")
img = img.resize((512, 512))

In [ ]:
prompt = "a photo of elmo stangind in front of a fiery pit"

with autocast("cuda"):
    image = pipe(prompt=prompt, image=img, strength=.75, guidance_scale=7.5).images[0]
    
image

#image.save("astronaut_rides_horse.png")

### Make a collage of 3 by 5


In [26]:
from PIL import Image
import os

file =  os.path.join("images", "elmo.jpg")
img = Image.open(file).convert("RGB")
img = img.resize((512, 512))

In [29]:
rows = 3
columns =5

actual_prompt = "A painting of elmo raising his arms standing in front of a fire"

prompts = ["{0} in the style of Greg Rutkowski",
           "{0} in the style of Salvador Dali",
           "{0} in the style of Japanese Anime",
           "{0} in the style of Vincent Van Gogh",
           "{0} in the style of Rembrandt",
           "{0} in the style of Edgar Degas",
           "{0} in the style of Pablo Picasso",
           "{0} in the style of Andy Wharhol",
           "{0} in the style of Georgia O'Keefe",
           "{0} in the style of Monet",
           "{0} in the style of Vermeer",
           "{0} in the style of Edvard Munch",
           "{0} in the style of Henri Matisse",
           "{0} in the style of Gustav Klimt",
           "{0} in the style of Edward Hopper"]

images = [""] * (rows*columns)

#for some reason it's bumping into the nsfw filter, nothing seems innapropiate so disabiling it
def dummy(images, **kwargs):
    return (images, [False])
pipe.safety_checker = dummy

for i in range(rows*columns):
    images[i] = pipe(prompt=prompts[i].format(actual_prompt), image=img, strength=.75, guidance_scale=7.5).images[0]

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

...

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

In [30]:
#ChatGPT wrote this
def create_collage(images, output_path, num_columns, num_rows):
    total_images = num_columns * num_rows

    # Check if the number of images matches the specified grid
    if len(images) != total_images:
        raise ValueError("Number of images doesn't match the grid size.")

    # Get the size of the first image
    image_width, image_height = images[0].size

    # Calculate collage size
    collage_width = num_columns * image_width
    collage_height = num_rows * image_height

    # Create a new blank image for the collage
    collage = Image.new('RGB', (collage_width, collage_height))

    # Paste each image onto the collage
    for i in range(num_rows):
        for j in range(num_columns):
            idx = i * num_columns + j
            collage.paste(images[idx], (j * image_width, i * image_height))

    # Save the collage
    collage.save(output_path)


create_collage(images, "images/collage_elmo.jpg", columns, rows)